In [44]:
import os
import time
import base64
from openai import OpenAI

In [46]:
API_KEY = "sk-proj-E1_iQA" #Provide your own KEY

In [48]:
IMAGE_DIR = './Bitwarden Photos/case 2' #your path to your screenshot to analyze
image_files = sorted([
    os.path.join(IMAGE_DIR, f)
    for f in os.listdir(IMAGE_DIR)
    if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))
])

client = OpenAI(api_key=API_KEY)


def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


# Load your persona from a text file
with open("PersonaA.txt", "r", encoding="utf-8") as persona_file:
    ux_persona = persona_file.read()

all_detailed_reviews = []
start_timer = time.time()

# === Step 1: Detailed Analysis Per Screenshot ===
for image_path in image_files:
    print(f"Processing {image_path}...")

    image_base64 = encode_image_to_base64(image_path)

    detailed_prompt = f"""
You are {ux_persona}.

Please analyze the attached screenshot of the Bitwarden app’s, does the password match on all views ? 

Now analyze this screenshot:
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": detailed_prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_base64}"}}
                ]
            }
        ],
        max_tokens=1000
    )

    review = f"Screenshot: {os.path.basename(image_path)}\n{response.choices[0].message.content}\n{'-'*60}\n"
    all_detailed_reviews.append(review)

# Save Detailed Feedback into MD file
detailed_report = "# UX Review Report\n\n## Detailed Screenshot Reviews\n\n"
for review in all_detailed_reviews:
    detailed_report += f"{review}\n"

with open("detailed_feedback.md", "w", encoding="utf-8") as file:
    file.write(detailed_report)

# Generate Final Summary Feedback that give an overview of all the feedback
feedback_prompt = f"""
{ux_persona}

Based on the detailed  reviews provided in the report below, generate comprehensive user feedback.
give me like an app review comment by sumurizing the detailed feedback, if the detailed_report include example put them in.

Here is the detailed feedback for analysis:

{detailed_report}
"""

feedback_response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": feedback_prompt}],
    max_tokens=1000
)

user_feedback = feedback_response.choices[0].message.content

# Save final feedback to markdown
with open("final_user_review.md", "w", encoding="utf-8") as file:
    file.write("# Final User Review\n\n")
    file.write(user_feedback)

end_timer = time.time()
execution_time = end_timer - start_timer

# === Final Execution Time ===
print("\n TASK COMPLETED:")
print(f"Detailed feedback saved as: detailed_feedback.md")
print(f"Saved as: final_user_review.md")
print(f"Total Execution Time: {execution_time:.2f} seconds")


Processing ./Bitwarden Photos/case 2/password comparison.png...

 TASK COMPLETED:
Detailed feedback saved as: detailed_feedback.md
Saved as: final_user_review.md
Total Execution Time: 26.35 seconds
